In [ ]:
#Import Library that used
import os
import json
import shutil
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras.models import load_model

In [ ]:
#Direktori data yang sudah di praprosesing
directory = "/content/drive/MyDrive/Capstone/data_oversampling"

In [ ]:
#Labeling Data
filepaths = []
labels = []

folds = os.listdir(directory)

for fold in folds:

    foldpath = os.path.join(directory, fold)
    filelist = os.listdir(foldpath)

    for file in filelist:

        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

Fseries = pd.Series(filepaths, name='Filepath')
Lseries = pd.Series(labels, name='Labels')

df = pd.concat([Fseries, Lseries], axis=1)

In [ ]:
#Split data
train_df, ts_df = train_test_split(df,test_size= 0.2, shuffle=True, random_state=42, stratify= df['Labels'])

test_df, valid_df = train_test_split(ts_df,test_size= 0.5,shuffle=True,random_state=42,
                                          stratify= ts_df['Labels'])

In [ ]:
#Check dataset and shape
print("Training set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)
print("Test set shape:", test_df.shape)

In [ ]:
#Resize image
batch_size = 32
img_size = (256,256)

gen = ImageDataGenerator(rescale=1./255)

train_gen = gen.flow_from_dataframe(train_df, x_col='Filepath', y_col='Labels',target_size= img_size,
                                   class_mode= 'categorical', color_mode='rgb', shuffle = True,
                                   batch_size= batch_size)

valid_gen = gen.flow_from_dataframe(valid_df, x_col='Filepath', y_col='Labels',target_size= img_size,
                                   class_mode= 'categorical', color_mode='rgb', shuffle = True,
                                   batch_size= batch_size)

test_gen = gen.flow_from_dataframe(test_df, x_col='Filepath', y_col='Labels',target_size= img_size,
                                   class_mode= 'categorical', color_mode='rgb', shuffle = False,
                                   batch_size= batch_size)

In [ ]:
#Model using TF
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape = (256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')

])

#Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Train model
model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=20,
    validation_data=valid_gen,
    validation_steps=valid_gen.samples // batch_size
)



In [ ]:
#Test model accuracy
test_loss, test_acc = model.evaluate(test_gen, verbose=2)
print('\nTest accuracy:', test_acc)
y_pred = model.predict(test_gen, steps=len(test_gen))
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_gen.classes
class_labels = list(test_gen.class_indices.keys())

#Confusion Matrix and F1-score
report = classification_report(y_true, y_pred_classes, target_names=class_labels)
print(report)

f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'Weighted F1 score: {f1}')

conf_matrix = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()